In [ ]:
import pandas as pd

In [ ]:
scales=['S_COVID_SYMPTOMS','S_COVID_COUGH','S_COVID_FEVER','S_COVID_BREATH','S_COVID_FATIGUE','S_COVID_PAIN','S_COVID_CONFUSION','S_COVID_TROUBLE','S_COVID_BLUISH','S_COVID_OVERALL','S_CORONA']

Read the survey file which will be used to obtain patient ids of patients who have suffered from covid-19 and the date of start of symptoms in case it is not available in the participants details file

In [ ]:
survey=pd.read_csv('/content/surveys.csv')

In [ ]:
survey.head()

,user_code,scale,created_at,value,text
0,01bad5a519,S_CORONA,2020-04-23,2,Symptoms are characteristic of coronavirus
1,01bad5a519,S_COVID_BLUISH,2020-04-23,1,User isn’t experiencing symptom
2,01bad5a519,S_COVID_BLUISH,2020-04-25,1,User isn’t experiencing symptom
3,01bad5a519,S_COVID_BLUISH,2020-04-27,1,User isn’t experiencing symptom
4,01bad5a519,S_COVID_BLUISH,2020-04-29,1,User isn’t experiencing symptom


Read the participants details file. We will need this to check symptoms onset against the patient id.

In [ ]:
participants=pd.read_csv('/content/participants.csv')
participants.head()

,user_code,gender,age_range,city,country,height,weight,symptoms_onset
0,007b8190cf,m,25-34,Mandalay,Myanmar,170.18,96.162,NaN
1,013f6d3e5b,f,18-24,São Paulo,Brazil,174.00,77.300,5/15/2020
2,01bad5a519,m,45-54,St Petersburg,Russia,178.00,92.000,4/5/2020
3,0210b20eea,f,25-34,Sochi,Russia,169.00,60.000,5/6/2020
4,024719e7da,f,45-54,St Petersburg,Russia,158.00,68.500,5/27/2020


Create a list of patients who have had covid using the scale in the survey.csv file. Also create a list of symptoms onset dates by choosing from the particpants.csv file, or from created_at field for the first observation in survey.csv. So the symptoms onset date at index i of the dates list corresponds to patient i in patients.

In [ ]:
dates=[]
patients=[]
for id in pd.unique(survey['user_code']):
  
  pat=survey[survey['user_code']==id].reset_index()
  if(pat.loc[0,'scale'] in scales):
    
    patients.append(id)
    pati=participants[participants['user_code']==id].reset_index(drop=True)
    if(pd.isna(pati.loc[0,'symptoms_onset'])):
      # print(pat.loc[0,'created_at'])
      dates.append(pd.to_datetime(pat.loc[0,'created_at']))
    else:
      dates.append(pd.to_datetime(pati['symptoms_onset']).values[0])
      # print(pati['symptoms_onset'].values[0])

print(dates) 


[numpy.datetime64('2020-04-05T00:00:00.000000000'), numpy.datetime64('2020-03-25T00:00:00.000000000'), numpy.datetime64('2020-04-01T00:00:00.000000000'), numpy.datetime64('2020-03-10T00:00:00.000000000'), numpy.datetime64('2020-03-27T00:00:00.000000000'), numpy.datetime64('2020-04-10T00:00:00.000000000'), numpy.datetime64('2020-04-19T00:00:00.000000000'), numpy.datetime64('1999-03-06T00:00:00.000000000'), numpy.datetime64('2020-04-11T00:00:00.000000000'), numpy.datetime64('2020-04-04T00:00:00.000000000'), numpy.datetime64('2020-03-14T00:00:00.000000000'), Timestamp('2020-04-23 00:00:00'), numpy.datetime64('2020-04-18T00:00:00.000000000'), numpy.datetime64('2020-04-26T00:00:00.000000000'), Timestamp('2020-04-14 00:00:00'), numpy.datetime64('2020-04-12T00:00:00.000000000'), Timestamp('2020-04-11 00:00:00'), numpy.datetime64('2020-04-22T00:00:00.000000000'), numpy.datetime64('2020-05-11T00:00:00.000000000'), numpy.datetime64('2020-04-01T00:00:00.000000000'), numpy.datetime64('2020-04-26T0

Check number of patients

In [ ]:
len(patients)

100

Read the hrv measurements file

In [ ]:
hrv=pd.read_csv('/content/hrv_measurements.csv')

In [ ]:
hrv.head()

,user_code,rr_code,measurement_datetime,time_of_day,bpm,meanrr,mxdmn,sdnn,rmssd,pnn50,mode,amo,lf,hf,vlf,lfhf,total_power,how_feel,how_mood,how_sleep,tags,rr_data
0,007b8190cf,10489a6aea,2020-04-21 21:23:08,morning,75,795.90,0.12,45.802,54.174,15.15,0.775,53.0,508.0,1076.0,267.0,0.472,1851.0,0,-1,NaN,COVID-19; Workout; Sex; Hobby; Studying; Sleep...,"819,1008,831,847,785,778,866,839,801,793,846,8..."
1,007b8190cf,9610d4d4dc,2020-04-26 11:19:25,morning,70,858.00,0.11,32.889,33.022,16.16,0.875,54.0,409.0,310.0,176.0,1.319,895.0,0,0,0.0,NaN,"888,775,811,883,890,894,894,899,893,889,890,83..."
2,013f6d3e5b,f3de056155,2020-05-15 04:14:21,night,83,724.10,0.17,54.811,65.987,17.17,0.725,46.0,432.0,881.0,194.0,0.490,1507.0,-1,-2,NaN,COVID-19; Fast/Diet; Hungry; Tired; Fever; I c...,"694,832,642,801,751,716,737,742,773,760,701,73..."
3,013f6d3e5b,b04489e32f,2020-05-19 03:06:02,night,75,802.64,0.20,72.223,70.039,22.22,0.825,43.0,814.0,1487.0,1719.0,0.547,4020.0,0,0,NaN,NaN,"821,817,771,805,833,788,747,724,792,825,775,75..."
4,01bad5a519,ac52c706c6,2019-12-31 09:07:43,morning,78,768.07,0.10,29.650,21.196,4.04,0.775,56.0,489.0,128.0,96.0,3.820,713.0,0,0,0.0,NaN,"741,740,734,737,740,731,751,747,745,728,747,76..."


Convert the meaurement_datetime field of hrv_measurements.csv and then extract only the date part.

In [ ]:
hrvcopy=hrv
hrvcopy['measurement_datetime']=pd.to_datetime(hrv['measurement_datetime']).dt.date

Convert the symptom onset dates to datetime type so that they can be compared with the dates in hrv measurements

In [ ]:
dates1=pd.to_datetime(dates)

In [ ]:
dates1

DatetimeIndex(['2020-04-05', '2020-03-25', '2020-04-01', '2020-03-10',
               '2020-03-27', '2020-04-10', '2020-04-19', '1999-03-06',
               '2020-04-11', '2020-04-04', '2020-03-14', '2020-04-23',
               '2020-04-18', '2020-04-26', '2020-04-14', '2020-04-12',
               '2020-04-11', '2020-04-22', '2020-05-11', '2020-04-01',
               '2020-04-26', '2020-03-08', '2020-04-19', '2020-04-22',
               '2020-04-14', '2022-04-21', '2020-03-26', '1984-04-05',
               '2020-05-01', '2020-03-22', '2020-05-10', '2020-03-17',
               '2020-05-08', '2020-04-15', '2020-04-16', '2020-04-04',
               '1982-12-16', '2020-04-12', '2020-05-03', '2020-03-16',
               '2020-03-21', '2020-04-04', '2020-04-01', '2020-04-06',
               '2020-03-30', '2020-03-28', '2021-04-01', '2020-04-08',
               '2020-03-28', '2020-05-12', '2020-04-07', '1967-01-30',
               '2020-03-24', '2020-04-25', '1993-01-01', '2020-04-04',
      

In [ ]:
patients

['01bad5a519',
 '05bcfae0a4',
 '0bdfbddb2b',
 '0f2d7af30c',
 '128d2c7888',
 '18c4ac7b38',
 '18f2738516',
 '1942df1c47',
 '1b9321b648',
 '1ce1d77659',
 '1ed25f66e9',
 '1fa680ad84',
 '224babc773',
 '257b51da6d',
 '276ab22485',
 '27fccfd61b',
 '285e0d75a6',
 '294bee940a',
 '295ed96279',
 '29d2a4bb3a',
 '2a6836ff3b',
 '2e94435085',
 '324912eca4',
 '35a6404d88',
 '3769e88689',
 '393bac62f4',
 '3acfbb328e',
 '3fff7627ca',
 '403155ae59',
 '42a99d8248',
 '43e1055eb8',
 '45cc34419c',
 '4985083f4d',
 '4c2702c3e1',
 '4c3311ade3',
 '4e1e508f56',
 '50249775be',
 '5108b04245',
 '520034e204',
 '5228cb9439',
 '522efc94b9',
 '5d200bd1c6',
 '5efbee746a',
 '6066fda6e3',
 '61a79c730c',
 '629e27a043',
 '638c757e73',
 '67fdb397e4',
 '6be5033971',
 '6ecfe4a351',
 '71980b2daf',
 '78cc008261',
 '7ba5381254',
 '7d2c87fb7e',
 '83161408c1',
 '856d41cc60',
 '866977681f',
 '89ec5a2500',
 '982ec78569',
 '9871ee5e7b',
 '997f44d61a',
 '9b4a5ded49',
 'a074b7fc98',
 'a1c2e6b2eb',
 'a4cebb51f2',
 'a62c11c164',
 'aa036185

Create a list of dates x, corresponding to the date 15 days before the onset of symptoms for each patient. This is to use any measurements before the dates in x as non-covid measurements. The 15 days gap is provided for the incubation period when symptoms are not visible.

In [ ]:
import datetime
x=[]
days = datetime.timedelta(15)
for date in dates1:
  x.append(date-days)
x

[Timestamp('2020-03-21 00:00:00'),
 Timestamp('2020-03-10 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-02-24 00:00:00'),
 Timestamp('2020-03-12 00:00:00'),
 Timestamp('2020-03-26 00:00:00'),
 Timestamp('2020-04-04 00:00:00'),
 Timestamp('1999-02-19 00:00:00'),
 Timestamp('2020-03-27 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-02-28 00:00:00'),
 Timestamp('2020-04-08 00:00:00'),
 Timestamp('2020-04-03 00:00:00'),
 Timestamp('2020-04-11 00:00:00'),
 Timestamp('2020-03-30 00:00:00'),
 Timestamp('2020-03-28 00:00:00'),
 Timestamp('2020-03-27 00:00:00'),
 Timestamp('2020-04-07 00:00:00'),
 Timestamp('2020-04-26 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-04-11 00:00:00'),
 Timestamp('2020-02-22 00:00:00'),
 Timestamp('2020-04-04 00:00:00'),
 Timestamp('2020-04-07 00:00:00'),
 Timestamp('2020-03-30 00:00:00'),
 Timestamp('2022-04-06 00:00:00'),
 Timestamp('2020-03-11 00:00:00'),
 Timestamp('1984-03-21 00:00:00'),
 Timestamp('2020-04-

Two folders for covid and non covid are created. Based on the date of measurement, hrv data for each patient is converted into a csv file for 15 time steps each and put into the respective folders. Read comments.

In [ ]:
folder1='/content/sample_data/Non_Covid'
folder2='/content/sample_data/Covid'


i=0
non=[] #these lists store the ids of patients for whom non covid data is available
cov=[] #these lists store the ids of patients for whom covid data is available

for id in patients:
  pat=hrvcopy[hrvcopy['user_code']==id].reset_index(drop=True) ## we create a new table for each user id and reset indices so they start with 0
  sample1=pat[pat['measurement_datetime']<=x[i]].reset_index(drop=True) #sample1 stores observations made on or before the date in x and hence are all non-covid measurements
  sample2=pat[pat['measurement_datetime']>=dates1[i]].reset_index(drop=True) #sample2 stores observations made on or after the date in dates1(date of onset in symptoms) and hence are all covid measurements
  if(len(sample1)>=15): #since we want measurements for 15 timesteps, we use the it only if its length is alteast 15
    non.append(id)
    n=len(sample1)
    div=n%15 
    no_of_obs=div+1 #no_of_obs tell us how many sets of 15 observations with overlapping windows we can obtain. For example if sample1 has 17 rows, 0-14 can be one set of observations, 1-15 another, 2-16, and finally 3-17. Giving us 15 continuous observations with 15 time steps each.
    for j in range(no_of_obs):
      df1=sample1.iloc[j:j+15,:].reset_index(drop=True) # we move down the table starting with j as starting index and take 15 observations from that index
      pd.DataFrame.to_csv(df1,folder1+'/'+id+str(j)+'.csv') # the selected 15 rows are added to a csv file with name userid+j and added to the non-covid folder
  
  if(len(sample2)>=15): # again check if there are atleast 15 observations
    cov.append(id)
    df2=sample2.iloc[:15,:].reset_index(drop=True) # we want hrv measurements starting from the day of onset of symptoms only, we don't create overlapping windows of meaurement. So we just use the first 15 measurements from the onset of symptoms
    pd.DataFrame.to_csv(df2,folder2+'/'+id+'.csv') #similarly store the data in csv files and since only one set of covid observations are collected per userid, we do not have to append a number j at the end to signify same user, different observations like we did previously

  i+=1

In [ ]:
non

['295ed96279',
 '4c2702c3e1',
 '638c757e73',
 'a1c2e6b2eb',
 'a62c11c164',
 'c174f32d88',
 'fcf3ea75b0']

In [ ]:
cov

['01bad5a519',
 '1942df1c47',
 '1ce1d77659',
 '276ab22485',
 '295ed96279',
 '2e94435085',
 '3769e88689',
 '3acfbb328e',
 '403155ae59',
 '4c2702c3e1',
 '5108b04245',
 '5228cb9439',
 '522efc94b9',
 '5efbee746a',
 '6be5033971',
 '71980b2daf',
 '78cc008261',
 '856d41cc60',
 '9871ee5e7b',
 'a1c2e6b2eb',
 'a62c11c164',
 'aa036185e3',
 'b325aa1406',
 'b9b65b7a69',
 'c174f32d88',
 'f9edcb7056',
 'fcf3ea75b0',
 'fde84801d8']

In [ ]:
!zip -r Covid.zip /content/sample_data/Covid

  adding: content/sample_data/Covid/ (stored 0%)
  adding: content/sample_data/Covid/522efc94b9.csv (deflated 58%)
  adding: content/sample_data/Covid/5228cb9439.csv (deflated 56%)
  adding: content/sample_data/Covid/a1c2e6b2eb.csv (deflated 63%)
  adding: content/sample_data/Covid/b325aa1406.csv (deflated 63%)
  adding: content/sample_data/Covid/1942df1c47.csv (deflated 56%)
  adding: content/sample_data/Covid/403155ae59.csv (deflated 58%)
  adding: content/sample_data/Covid/9871ee5e7b.csv (deflated 57%)
  adding: content/sample_data/Covid/856d41cc60.csv (deflated 58%)
  adding: content/sample_data/Covid/71980b2daf.csv (deflated 58%)
  adding: content/sample_data/Covid/1ce1d77659.csv (deflated 58%)
  adding: content/sample_data/Covid/fde84801d8.csv (deflated 59%)
  adding: content/sample_data/Covid/276ab22485.csv (deflated 58%)
  adding: content/sample_data/Covid/3769e88689.csv (deflated 59%)
  adding: content/sample_data/Covid/f9edcb7056.csv (deflated 54%)
  adding: content/sample_da

In [ ]:
!zip -r Non_Covid.zip /content/sample_data/Non_Covid

  adding: content/sample_data/Non_Covid/ (stored 0%)
  adding: content/sample_data/Non_Covid/a1c2e6b2eb10.csv (deflated 60%)
  adding: content/sample_data/Non_Covid/4c2702c3e14.csv (deflated 60%)
  adding: content/sample_data/Non_Covid/fcf3ea75b06.csv (deflated 57%)
  adding: content/sample_data/Non_Covid/fcf3ea75b09.csv (deflated 58%)
  adding: content/sample_data/Non_Covid/c174f32d885.csv (deflated 61%)
  adding: content/sample_data/Non_Covid/a62c11c1645.csv (deflated 57%)
  adding: content/sample_data/Non_Covid/a1c2e6b2eb4.csv (deflated 60%)
  adding: content/sample_data/Non_Covid/fcf3ea75b013.csv (deflated 58%)
  adding: content/sample_data/Non_Covid/fcf3ea75b010.csv (deflated 58%)
  adding: content/sample_data/Non_Covid/a1c2e6b2eb2.csv (deflated 59%)
  adding: content/sample_data/Non_Covid/295ed962790.csv (deflated 59%)
  adding: content/sample_data/Non_Covid/a1c2e6b2eb9.csv (deflated 61%)
  adding: content/sample_data/Non_Covid/295ed962793.csv (deflated 59%)
  adding: content/sam